# Notebook to explore CNA, methylation and mutation data, see matchings

## Methylation data

In [1]:
import pandas as pd
import numpy as np

In [2]:
full_methylation = pd.read_csv("../data/raw/d82e2c44-89eb-43d9-b6d3-712732bf6a53.tsv", sep="\t")

In [3]:
full_methylation_trans = full_methylation.set_index("Composite Element REF").T

In [4]:
full_methylation_trans = full_methylation_trans.reset_index()

In [7]:
full_methylation_trans.head(5)

Composite Element REF,index,cg00000292,cg00003994,cg00005847,cg00007981,cg00008493,cg00008713,cg00009407,cg00011459,cg00012199,...,cg27651218,cg27652350,cg27653134,cg27654142,cg27655905,cg27657283,cg27661264,cg27662379,cg27662877,cg27665659
0,TCGA-02-0001-01C-01D-0186-05,0.846378,0.066406,0.330927,0.026121,0.925941,0.060554,0.058225,0.869235,0.048453,...,0.926899,0.354071,0.924303,0.060605,0.064736,0.029954,0.355380,0.061701,0.068880,0.044622
1,TCGA-02-0003-01A-01D-0186-05,0.732044,0.131451,0.839389,0.039102,0.934192,0.165144,0.081359,0.727453,0.045299,...,0.897646,0.689550,0.919943,0.042318,0.057933,0.073895,0.361591,0.051867,0.065503,0.053852
2,TCGA-02-0006-01B-01D-0186-05,0.678917,0.089675,0.469196,0.021105,0.924509,0.061625,0.081596,0.898071,0.048361,...,0.906227,0.037738,0.922556,0.064637,0.072039,0.051412,0.417782,0.055937,0.062774,0.097196
3,TCGA-02-0007-01A-01D-0186-05,0.846951,0.040882,0.355053,0.039487,0.930292,0.060462,0.103439,0.873396,0.046849,...,0.930233,0.239067,0.906665,0.058250,0.096103,0.122936,0.446803,0.058933,0.054686,0.052628
4,TCGA-02-0009-01A-01D-0186-05,0.855192,0.066100,0.605179,0.025252,0.926921,0.061412,0.076623,0.920929,0.047368,...,0.896734,0.098433,0.939404,0.060082,0.080667,0.029166,0.448412,0.054502,0.056168,0.048547


In [8]:
full_methylation_trans.sort_values(by="index", inplace=True)

In [9]:
full_methylation_trans["index"] = full_methylation_trans["index"].apply(lambda x: x[:12])

In [10]:
full_methylation_trans.drop_duplicates(subset="index", keep="first", inplace=True)

In [11]:
full_methylation_trans["index"].value_counts().head(5)

TCGA-AB-2863    1
TCGA-HL-7533    1
TCGA-A3-A8CQ    1
TCGA-RC-A7S9    1
TCGA-32-4210    1
Name: index, dtype: int64

## Load BRCA data too

In [8]:
tcga_brca_train = pd.read_pickle("../data/tcga_brca_raw_19036_train.pkl")
tcga_brca_test = pd.read_pickle("../data/tcga_brca_raw_19036_test.pkl")

In [9]:
tcga_brca_train_id = tcga_brca_train["tcga_id"]
tcga_brca_test_id = tcga_brca_test["tcga_id"]

tcga_brca_all_id = pd.concat([tcga_brca_train_id, tcga_brca_test_id])

In [10]:
tcga_all_n_samples = len(tcga_brca_all_id)
tcga_train_n_samples = len(tcga_brca_train_id)
tcga_test_n_samples = len(tcga_brca_test_id)
methy_n_samples = len(full_methylation_trans["index"])

NameError: name 'full_methylation_trans' is not defined

In [15]:
cond = tcga_brca_all_id.isin(full_methylation_trans['index'])
overlapping_all = len(tcga_brca_all_id[cond])
overlapping_all

1032

In [16]:
cond_train = tcga_brca_train_id.isin(full_methylation_trans['index'])
overlapping_train = len(tcga_brca_train_id[cond_train])

cond_test = tcga_brca_test_id.isin(full_methylation_trans['index'])
overlapping_test = len(tcga_brca_test_id[cond_test])

In [17]:
print("We have {}/{} samples with methylation data.\n Out of these:\n {}/{} on the training set\n {}/{} on the test set".format(overlapping_all, tcga_all_n_samples, overlapping_train, tcga_train_n_samples, overlapping_test, tcga_test_n_samples))

We have 1032/1053 samples with methylation data.
 Out of these:
 811/817 on the training set
 221/236 on the test set


### Which are the non-overlapping?

#### On the training set - 6 samples

In [18]:
tcga_brca_train_missing = tcga_brca_train_id[~cond_train]
tcga_brca_train_missing

122    TCGA-BH-A0B6
165    TCGA-A8-A084
325    TCGA-A8-A09K
547    TCGA-BH-A1F5
566    TCGA-D8-A146
726    TCGA-A8-A08S
Name: tcga_id, dtype: object

#### On the test set - 15 samples

In [19]:
tcga_brca_test_missing = tcga_brca_test_id[~cond_test]
tcga_brca_test_missing

14     TCGA-A7-A56D
120    TCGA-AR-A5QM
138    TCGA-BH-A5IZ
139    TCGA-BH-A5J0
161    TCGA-E9-A5FK
162    TCGA-E9-A5FL
173    TCGA-HN-A2OB
189    TCGA-OK-A5Q2
190    TCGA-OL-A5D6
191    TCGA-OL-A5D7
192    TCGA-OL-A5D8
193    TCGA-OL-A5DA
209    TCGA-PE-A5DC
210    TCGA-PE-A5DD
211    TCGA-PE-A5DE
Name: tcga_id, dtype: object

### Let's check the subtype of the non-overlapping ones

#### On the training set

In [21]:
[tcga_brca_train[tcga_brca_train["tcga_id"] == x]["Ciriello_subtype"] for x in tcga_brca_train_missing]

[122    LumA
 Name: Ciriello_subtype, dtype: object, 165    LumB
 Name: Ciriello_subtype, dtype: object, 325    LumA
 Name: Ciriello_subtype, dtype: object, 547    LumA
 Name: Ciriello_subtype, dtype: object, 566    LumA
 Name: Ciriello_subtype, dtype: object, 726    LumB
 Name: Ciriello_subtype, dtype: object]

#### On the test set

In [22]:
[tcga_brca_test[tcga_brca_test["tcga_id"] == x]["subtype"] for x in tcga_brca_test_missing]

[14    LumA
 Name: subtype, dtype: object, 120    LumA
 Name: subtype, dtype: object, 138    Basal
 Name: subtype, dtype: object, 139    LumA
 Name: subtype, dtype: object, 161    LumA
 Name: subtype, dtype: object, 162    Basal
 Name: subtype, dtype: object, 173    LumA
 Name: subtype, dtype: object, 189    LumA
 Name: subtype, dtype: object, 190    Normal
 Name: subtype, dtype: object, 191    Basal
 Name: subtype, dtype: object, 192    LumA
 Name: subtype, dtype: object, 193    LumA
 Name: subtype, dtype: object, 209    LumA
 Name: subtype, dtype: object, 210    LumA
 Name: subtype, dtype: object, 211    LumA
 Name: subtype, dtype: object]

<br>

### The overlapping samples are of reasonable amount, and the missing subtypes are mostly from the classes that have the higher amount of samples, which means that losing this number of samples should not be a big problem

<br>
<br>
<br>

## CNA data

In [2]:
full_cna = pd.read_csv("../data/raw/data_CNA.txt", sep="\t")

In [3]:
full_cna.head(5)

,Hugo_Symbol,Entrez_Gene_Id,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5T-A9QA-01,TCGA-A1-A0SB-01,...,TCGA-UL-AAZ6-01,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01
0,ACAP3,116983.0,0,-1,-1,0,0,0,-1,0,...,0,-1,0,0,-1,-1,0,-1,-1,0
1,ACTRT2,140625.0,0,-1,-1,0,0,0,-1,0,...,0,-1,0,0,-1,-1,0,-1,-1,0
2,AGRN,375790.0,0,-1,-1,0,0,0,-1,0,...,0,-1,0,0,-1,-1,0,-1,-1,0
3,ANKRD65,441869.0,0,-1,-1,0,0,0,-1,0,...,0,-1,0,0,-1,-1,0,-1,-1,0
4,ATAD3A,55210.0,0,-1,-1,0,0,0,-1,0,...,0,-1,0,0,-1,-1,0,-1,-1,0


In [4]:
full_cna.drop(["Entrez_Gene_Id"], axis=1, inplace=True)
full_cna_t = full_cna.set_index("Hugo_Symbol").T
full_cna_t = full_cna_t.reset_index()

In [5]:
full_cna_t.head(5)

Hugo_Symbol,index,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,TCGA-3C-AAAU-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,TCGA-3C-AALI-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,1,1,1,-1
2,TCGA-3C-AALJ-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
3,TCGA-3C-AALK-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TCGA-4H-AAAK-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
full_cna_t["index"] = full_cna_t["index"].apply(lambda x: x[:12])

In [19]:
cond_train_cna = tcga_brca_train_id.isin(full_cna_t["index"])
overlapping_train_cna = len(tcga_brca_train_id[cond_train_cna])

cond_test_cna = tcga_brca_test_id.isin(full_cna_t["index"])
overlapping_test_cna = len(tcga_brca_test_id[cond_test_cna])

In [20]:
print("We have {}/{} samples with methylation data.\n Out of these:\n {}/{} on the training set\n {}/{} on the test set".format(overlapping_train_cna+overlapping_test_cna, tcga_all_n_samples, overlapping_train_cna, tcga_train_n_samples, overlapping_test_cna, tcga_test_n_samples))

We have 1048/1053 samples with methylation data.
 Out of these:
 812/817 on the training set
 236/236 on the test set


### What are the missing samples?

#### On the training set - 5 samples

In [23]:
tcga_brca_train_id[~cond_train_cna]

122    TCGA-BH-A0B6
165    TCGA-A8-A084
325    TCGA-A8-A09K
566    TCGA-D8-A146
726    TCGA-A8-A08S
Name: tcga_id, dtype: object

#### On the test set - NONE

In [24]:
tcga_brca_test_id[~cond_test_cna]

Series([], Name: tcga_id, dtype: object)